In [1]:
%matplotlib widget

import logging
import time

import cftime
import httpx
import matplotlib.pyplot as plt
import netCDF4
import pandas as pd
import shapely.io
import sqlmodel
from sqlalchemy import (
    bindparam, 
    func
)

from arpav_ppcv import (
    database as db,
    operations,
)
from arpav_ppcv.config import get_settings
from arpav_ppcv.schemas.base import (
    CoverageDataSmoothingStrategy,
    ObservationDataSmoothingStrategy,
    ObservationAggregationType,
    Season,
)
from arpav_ppcv.schemas import coverages
from arpav_ppcv.schemas.coverages import CoverageInternal
from arpav_ppcv.schemas import observations

logging.basicConfig(level=logging.DEBUG)
logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("httpcore").setLevel(logging.WARNING)
logging.getLogger("matplotlib").setLevel(logging.WARNING)

settings = get_settings()
session = sqlmodel.Session(db.get_engine(settings))
http_client = httpx.Client()

In [2]:
cov = db.get_coverage(session, "tdd_annual_seasonal-annual-historical-tdd-winter-absolute")

In [7]:
cov.configuration.color_scale_min, cov.configuration.color_scale_max

(-5.0, 20.0)

In [3]:
from arpav_ppcv import palette

In [5]:
parsed_palette = palette.parse_palette(cov.configuration.palette, settings.palettes_dir)

In [6]:
palette.apply_palette(parsed_palette, cov.configuration.color_scale_min, cov.configuration.color_scale_max)

[(-5.0, '#FFffffcc'),
 (-1.875, '#FFffeda0'),
 (1.25, '#FFfed976'),
 (4.375, '#FFfeb24c'),
 (7.5, '#FFfd8d3c'),
 (10.625, '#FFfc4e2a'),
 (13.75, '#FFe31a1c'),
 (16.875, '#FFbd0026'),
 (20.0, '#FF800026')]

In [11]:
cov.configuration.get_thredds_url_fragment(cov.identifier)

'cline_yr/TDd_S01_*.nc'